# Algorithm Implement


**目录：**
1. 算法分析

2. agent结构

3. loss计算

4. 训练过程

---

## 1) 算法分析

首先，我们对比2013年和2015年两篇paper的算法的不同之处

<img src="./imgs/Old DQN.png"  width="550" height="500" align="left" />
<img src="./imgs/Better DQN.png"  width="400" height="500" align="right" />

一个很重要的区别是2015年的版本中引入了target action_value function $\hat{Q}$ 以及其独立的参数$\theta^-$，而且$\theta^-$的更新频率比$\theta$低很多。这个方法能帮助训练中的target更加稳定，从而提升训练的效果，我们也将直接实现2015年的算法

## 2) agent结构

### 1) 功能分析

首先我们应该考虑我们需要一个怎样的agent。

在算法中，可以看到agent需要做的事情有：
  * 选择action，随机或非随机
  * 更新$Q$的参数$\theta$
  * 更新$\hat{Q}$的参数$\theta^-$
  
而其他的一些部分与agent相关，但并不一定要融入agent内部：
  * 数据预处理
  * Replay Buffer
  
在这次的代码中，我们选择将**Replay Buffer**部分放入agent内部，这并不是必须的，但是可以让代码稍微简单一些。做出这样的选择后，我们可以大致确定agent类的主要部分：

  * 初始化，`agent.__init__(self, *args, **kwargs)`
    * 设置各种参数，如学习率，$\gamma$
    * 初始化Q-Network $Q, \hat{Q}$
    * 初始化Replay Buffer
  * 选择行动，`agent.act(self, state, eps)`
    * 随机选择
    * 非随机选择
  * 更新网络参数，`agent.learn(self)`
    * 从Replay Buffer中抽取mini-batch
    * 计算loss
    * 更新参数$\theta$
    * 如果需要，更新参数$\theta^-$
    
下面我们一个一个部分分析

### 2) 初始化

这个部分比较简单，只需要合理的传参就可以完成参数的初始化和网络的初始化，需要考虑的部分有：
  * 如何让两个网络权重一致
    * 方法1：使用`state_dict`方法，由$Q$进行`save`，然后`load`到$\hat{Q}$
    * 方法2：对每个layer的`data`进行提取，然后进行计算，最后使用`copy_`方法导入$\hat{Q}$
  * 如何设置Replay Buffer
    * 方法1：直接使用`collections`的`deque`类，只做储存功能
    * 方法2：在方法1的基础上，拓展成一个类，支持写入和抽样方法
    
**调整网络权重**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Q_Network(nn.Module):

    def __init__(self, state_size, action_size, hidden=[64, 64]):
        super(Q_Network, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0], hidden[1])
        self.fc3 = nn.Linear(hidden[1], action_size)

    def forward(self, state):
        x = state
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [2]:
#state_dict方法
net1 = Q_Network(5, 5, [3, 3])
net2 = Q_Network(5, 5, [3, 3])
net1.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.0679,  0.4332,  0.0654,  0.0443,  0.4194],
                      [-0.3549, -0.0235, -0.0443,  0.2450,  0.2451],
                      [-0.1684, -0.2973,  0.1277,  0.2525, -0.1271]])),
             ('fc1.bias', tensor([ 0.1053,  0.4053, -0.2949])),
             ('fc2.weight', tensor([[-0.2643, -0.4360, -0.5264],
                      [-0.0725, -0.4220,  0.5070],
                      [ 0.0573,  0.0750, -0.1777]])),
             ('fc2.bias', tensor([-0.0779,  0.0140,  0.3798])),
             ('fc3.weight', tensor([[ 0.4608,  0.0406, -0.0353],
                      [-0.0706,  0.3639,  0.2261],
                      [ 0.4154, -0.0203,  0.0955],
                      [-0.4545, -0.1354, -0.3108],
                      [ 0.1875, -0.2246, -0.2300]])),
             ('fc3.bias',
              tensor([ 0.4948,  0.0286,  0.3151, -0.4653,  0.3695]))])

In [3]:
net2.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-0.2910,  0.3765, -0.0545, -0.0391, -0.3392],
                      [-0.3499, -0.3084, -0.2932, -0.2775,  0.2621],
                      [-0.2316, -0.0450, -0.4080, -0.1062, -0.1366]])),
             ('fc1.bias', tensor([ 0.2956,  0.1464, -0.3279])),
             ('fc2.weight', tensor([[ 0.1493, -0.3801,  0.0662],
                      [ 0.2155, -0.3814,  0.3031],
                      [ 0.1180, -0.0903,  0.2089]])),
             ('fc2.bias', tensor([0.5276, 0.3552, 0.2416])),
             ('fc3.weight', tensor([[-0.2751, -0.3567,  0.5149],
                      [ 0.5676,  0.2579,  0.5266],
                      [ 0.1522, -0.1482,  0.4931],
                      [ 0.0868, -0.3292, -0.5639],
                      [ 0.4660, -0.3056,  0.1159]])),
             ('fc3.bias',
              tensor([ 0.0590, -0.4560,  0.1501, -0.1528,  0.0850]))])

In [4]:
torch.save(net1.state_dict(), 'sample_weight.pth')
net2.load_state_dict(torch.load('sample_weight.pth'))
net2.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.0679,  0.4332,  0.0654,  0.0443,  0.4194],
                      [-0.3549, -0.0235, -0.0443,  0.2450,  0.2451],
                      [-0.1684, -0.2973,  0.1277,  0.2525, -0.1271]])),
             ('fc1.bias', tensor([ 0.1053,  0.4053, -0.2949])),
             ('fc2.weight', tensor([[-0.2643, -0.4360, -0.5264],
                      [-0.0725, -0.4220,  0.5070],
                      [ 0.0573,  0.0750, -0.1777]])),
             ('fc2.bias', tensor([-0.0779,  0.0140,  0.3798])),
             ('fc3.weight', tensor([[ 0.4608,  0.0406, -0.0353],
                      [-0.0706,  0.3639,  0.2261],
                      [ 0.4154, -0.0203,  0.0955],
                      [-0.4545, -0.1354, -0.3108],
                      [ 0.1875, -0.2246, -0.2300]])),
             ('fc3.bias',
              tensor([ 0.4948,  0.0286,  0.3151, -0.4653,  0.3695]))])

In [5]:
# use .data and .data.copy_
def soft_update(net1, net2, tau):
    '''
    This function update net2's weights as weighted average of net1 and net2's weights
    net2 new weight = tau * net1 weight + (1 - tau) * net2 weight
    '''
    for target_param, local_param in zip(net2.parameters(), net1.parameters()):
        target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [6]:
net1 = Q_Network(5, 5, [3, 3])
net2 = Q_Network(5, 5, [3, 3])
net1.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.4417, -0.1202, -0.1739,  0.0483, -0.2992],
                      [ 0.3343, -0.4089, -0.0126,  0.1113,  0.3903],
                      [ 0.1090,  0.1604, -0.2945, -0.4120, -0.3667]])),
             ('fc1.bias', tensor([-0.2332,  0.2572, -0.1903])),
             ('fc2.weight', tensor([[ 0.4999,  0.4841,  0.0952],
                      [-0.4776,  0.1968,  0.0973],
                      [-0.2435,  0.2508, -0.2731]])),
             ('fc2.bias', tensor([ 0.0204, -0.4823,  0.2934])),
             ('fc3.weight', tensor([[-0.0859,  0.1219, -0.4255],
                      [ 0.2852,  0.5614, -0.4320],
                      [ 0.4332, -0.3744,  0.0568],
                      [-0.1654, -0.3326, -0.4354],
                      [ 0.4881, -0.0776,  0.0331]])),
             ('fc3.bias',
              tensor([-0.4016, -0.4265,  0.3851, -0.5375,  0.2060]))])

In [7]:
net2.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.4281,  0.3780,  0.0656,  0.0403, -0.1434],
                      [-0.1568, -0.0412,  0.1549,  0.2433,  0.0091],
                      [ 0.0028,  0.3524, -0.4089, -0.0479,  0.3002]])),
             ('fc1.bias', tensor([ 0.0923,  0.3289, -0.0749])),
             ('fc2.weight', tensor([[ 0.5554,  0.0996, -0.0194],
                      [-0.5737, -0.3324,  0.4286],
                      [ 0.1886,  0.1219,  0.2857]])),
             ('fc2.bias', tensor([-0.4790,  0.3780,  0.1038])),
             ('fc3.weight', tensor([[ 0.2041, -0.0458,  0.1566],
                      [ 0.4216, -0.3038, -0.2483],
                      [ 0.1693,  0.1799, -0.5065],
                      [-0.5575, -0.4304, -0.2116],
                      [-0.5322, -0.3069, -0.4577]])),
             ('fc3.bias',
              tensor([ 0.4317, -0.0374,  0.2507,  0.0995, -0.2147]))])

In [8]:
soft_update(net1, net2, 0.5)
net2.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.4349,  0.1289, -0.0541,  0.0443, -0.2213],
                      [ 0.0888, -0.2250,  0.0711,  0.1773,  0.1997],
                      [ 0.0559,  0.2564, -0.3517, -0.2300, -0.0332]])),
             ('fc1.bias', tensor([-0.0705,  0.2930, -0.1326])),
             ('fc2.weight', tensor([[ 0.5277,  0.2918,  0.0379],
                      [-0.5256, -0.0678,  0.2630],
                      [-0.0275,  0.1863,  0.0063]])),
             ('fc2.bias', tensor([-0.2293, -0.0522,  0.1986])),
             ('fc3.weight', tensor([[ 0.0591,  0.0381, -0.1344],
                      [ 0.3534,  0.1288, -0.3401],
                      [ 0.3012, -0.0972, -0.2248],
                      [-0.3615, -0.3815, -0.3235],
                      [-0.0221, -0.1923, -0.2123]])),
             ('fc3.bias',
              tensor([ 0.0151, -0.2319,  0.3179, -0.2190, -0.0043]))])

In [9]:
soft_update(net1, net2, 1)
net2.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.4417, -0.1202, -0.1739,  0.0483, -0.2992],
                      [ 0.3343, -0.4089, -0.0126,  0.1113,  0.3903],
                      [ 0.1090,  0.1604, -0.2945, -0.4120, -0.3667]])),
             ('fc1.bias', tensor([-0.2332,  0.2572, -0.1903])),
             ('fc2.weight', tensor([[ 0.4999,  0.4841,  0.0952],
                      [-0.4776,  0.1968,  0.0973],
                      [-0.2435,  0.2508, -0.2731]])),
             ('fc2.bias', tensor([ 0.0204, -0.4823,  0.2934])),
             ('fc3.weight', tensor([[-0.0859,  0.1219, -0.4255],
                      [ 0.2852,  0.5614, -0.4320],
                      [ 0.4332, -0.3744,  0.0568],
                      [-0.1654, -0.3326, -0.4354],
                      [ 0.4881, -0.0776,  0.0331]])),
             ('fc3.bias',
              tensor([-0.4016, -0.4265,  0.3851, -0.5375,  0.2060]))])

**Replay Buffer**

`collections`的`deque`只有一个主要参数，就是`maxlen`，也就是最大储存数量

`deque`和`list`几乎完全一致，只是会自动控制最大元素个数，所以很好使用，也很适合作为Replay Buffer使用

In [10]:
from collections import deque

test = deque(maxlen=5)
for i in range(10):
    test.append(i)
    print(test)

deque([0], maxlen=5)
deque([0, 1], maxlen=5)
deque([0, 1, 2], maxlen=5)
deque([0, 1, 2, 3], maxlen=5)
deque([0, 1, 2, 3, 4], maxlen=5)
deque([1, 2, 3, 4, 5], maxlen=5)
deque([2, 3, 4, 5, 6], maxlen=5)
deque([3, 4, 5, 6, 7], maxlen=5)
deque([4, 5, 6, 7, 8], maxlen=5)
deque([5, 6, 7, 8, 9], maxlen=5)


## 3) loss计算

### 1) `torch.gather`

此处要强调loss计算是因为，我们的网络对于输入$s_t$，会直接输出所有的$$Q(s_t, a_1), Q(s_t, a_2), ..., Q(s_t, a_n)$$
为了将一个batch中需要的Q-values便捷的提取出来，需要介绍一个一个函数`torch.gather`

`torch.gather`的一般用法为`torch.gather(input, dim, index)`，这个函数主要的功能是根据`dim`和`index`来提取`input`中的值，具体的来说：
  * `input`和`index`必须拥有同样的size，$(d_0, d_1, ..., d_k)$，除了`dim`维度
  * `dim`的范围不能超过size的长度，即上一行的$k$
  * `input`:一个`torch.tensor`对象，取值的来源
  * `dim`:一个整数，被取代的维度，之后补充具体意义
  * `index`:一个`torch.tensor`对象，必须是`torch.long`数据类型，用来指定提取的坐标
  * 数学表达式来看，函数返回值仍是一个$(d_0, d_1, ..., d_k)$维度，和`input`有相同数据类型的`torch.tensor`，不过在$[i_0, i_1,...,i_k]$坐标上的值是`input`对应坐标$[i_0, i_1, ...,i_{dim-1}, index[i_0, i_1,...,i_k], i_{dim+1},...,i_k]$
  * 简单来说，就是用`index`的值去改变原坐标中`dim`维度的值，这也是`index`需要是`torch.long`类型的原因

In [11]:
# example
test = torch.rand((5, 4))
test

tensor([[0.9755, 0.8373, 0.5262, 0.0664],
        [0.6652, 0.8752, 0.8146, 0.1734],
        [0.6196, 0.9246, 0.9948, 0.4292],
        [0.1522, 0.2403, 0.1694, 0.5246],
        [0.8555, 0.4947, 0.6869, 0.8264]])

In [12]:
index = torch.tensor([[1,2,2,0],
                      [3,1,1,2],
                      [0,0,0,0],
                      [1,1,1,1],
                      [2,2,2,2]], dtype=torch.long)

In [13]:
torch.gather(test, 0, index)

tensor([[0.6652, 0.9246, 0.9948, 0.0664],
        [0.1522, 0.8752, 0.8146, 0.4292],
        [0.9755, 0.8373, 0.5262, 0.0664],
        [0.6652, 0.8752, 0.8146, 0.1734],
        [0.6196, 0.9246, 0.9948, 0.4292]])

In [14]:
torch.gather(test, 1, index)

tensor([[0.8373, 0.5262, 0.5262, 0.9755],
        [0.1734, 0.8752, 0.8752, 0.8146],
        [0.6196, 0.6196, 0.6196, 0.6196],
        [0.2403, 0.2403, 0.2403, 0.2403],
        [0.6869, 0.6869, 0.6869, 0.6869]])

### 2) `torch.max`

`torch.max`比较简单，主要的使用方法是`torch.max(input, dim)`，作用就是沿着`dim`维度提取每个slice的最大值即对应下标，由于函数本身比较简单，就不单独给出样例了

下面我们进行两个具体的操作：
  * 提取`test`每一行的最大值
  * 提取`test`对应行的第(2,2,1,3,0)元素

In [15]:
#max
values, indices = torch.max(test, dim=1)
values, indices

(tensor([0.9755, 0.8752, 0.9948, 0.5246, 0.8555]), tensor([0, 1, 2, 3, 0]))

In [16]:
index = torch.tensor([[2], 
                      [2], 
                      [1], 
                      [3], 
                      [0]], dtype=torch.long)
torch.gather(test, 1, index)

tensor([[0.5262],
        [0.8146],
        [0.9246],
        [0.5246],
        [0.8555]])

In [17]:
index = torch.tensor([2, 2, 1, 3, 0], dtype=torch.long)
torch.gather(test, 1, index)

RuntimeError: invalid argument 4: Index tensor must have same dimensions as input tensor at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:638

由这两个函数，我们就可以计算loss了，具体的过程为：
  * 已知mini-batch中的state, action, next_state, reward, done
  * $Q$网络对应`net_local`，$\hat{Q}$网络对应`net_target`
  * 计算Q-values `Q_values = net_local(state)`
  * 根据action，提取对应的Q-values `Q_values = torch.gather(Q_values, 1, action)`
  * 计算next_state对应的最大的target values `Q_targets, _ = torch.max(net_target(next_state), 1, keepdim=True)`
  * 在target values中加入reward和done的信息 `Q_targets = reward + (1 - done) * Q_targets`
  * 注意`Q_targets`不应该有导数
  * 计算loss `loss = (Q_values - Q_targets).pow(2).mean()`
  * 使用`.backward()`方法更新参数
  
## 4) 训练过程

### 1) 结构分析
之前的notebook中我们已经示范了如何获得随机策略的reward平均值，我们将在这个的基础上扩充代码，将其变为可以用做训练的代码

In [ ]:
#env = gym.make()
num_episode = 5
max_t = 1000
reward_log = []

for _ in range(num_episode):
    
    # initialize
    env.reset()
    t = 0
    episodic_reward = 0
    
    for t in range(max_t):
        
        #env.render()
        action = env.action_space.sample() # random action
        _, reward, done, _ = env.step(action)
        episodic_reward += reward
        if done:
            break
    
    reward_log.append(episodic_reward)

在之前的代码中，我们已经完成的部分有：
  * 初始化参数
  * 初始化environment
  * 选择action，记录reward
 
现在我们需要增加的部分有：
  * 初始化和调整$\epsilon$的值以实现$\epsilon$-Greedy
  * agent选择action部分
  * agent的训练部分，即更新$Q$和$\hat{Q}$的对应参数
  * 监督训练进度的部分

之前我们已经分析了`agent`的结构，如果我们认为我们已经完成了`agent`类的设计，那么具体的任务就是：
  * 初始化和调整$\epsilon$的值以实现$\epsilon$-Greedy
  * 在合适的位置插入`agent.act()`
  * 在合适的位置插入`agent.learn()`
  * 监督训练进度的部分
  
## 2) 具体实现

In [ ]:
#env = gym.make()
num_episode = 5
max_t = 1000
reward_log = []
average_log = [] # monitor training process
eps = 1
eps_decay = 0.995
eps_min = 0.01
C = 4 # update weights every C steps

for i in range(num_episode):
    
    # initialize 
    state = env.reset()
    # preprocessing of state if necessary
    t = 0
    done = False
    episodic_reward = 0
    
    for t in range(max_t):
        
        action = agent.act(state, eps)
        next_state, reward, done, _ = env.step(action)
        # preprocessing of next_state if necessary
        episodic_reward += reward
        
        if t % C == 0:
            agent.learn()
            agent.soft_update()
        
        state = next_state.copy()
        if done:
            break
            
    eps = max(eps_min, eps*eps_decay)
            
    reward_log.append(episodic_reward)
    
    # monitor
    average_log.append(np.mean(rewards_log[-100:])) 
    print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}'.format(i, episodic_reward, average_log[-1]), end='')
    if i % 50 == 0:
        print()